<a href="https://colab.research.google.com/github/LuisCGit/HipHopPopularity/blob/main/pre_trained_ssast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install timm==0.4.5 pytorch-lightning optuna

     |████████████████████████████████| 287 kB 7.1 MB/s 
     |████████████████████████████████| 584 kB 59.3 MB/s 
     |████████████████████████████████| 308 kB 69.1 MB/s 
     |████████████████████████████████| 596 kB 57.0 MB/s 
     |████████████████████████████████| 136 kB 74.8 MB/s 
     |████████████████████████████████| 408 kB 72.3 MB/s 
     |████████████████████████████████| 1.1 MB 50.5 MB/s 
     |████████████████████████████████| 81 kB 11.8 MB/s 
     |████████████████████████████████| 210 kB 75.3 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
     |████████████████████████████████| 144 kB 12.3 MB/s 
     |████████████████████████████████| 78 kB 9.0 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 113 kB 76.5 MB/s 
     |████████████████████████████████| 146 kB 75.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11

In [4]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, r2_score, mean_absolute_error, mean_squared_error
import torch
import torch.nn as nn
from torch.nn import functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import optuna
from drive.MyDrive.MIT.HODL.ssast.src.models.ast_models import ASTModel

In [56]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchaudio import transforms

In [5]:
!wget -O mel_spec_datadict https://www.dropbox.com/s/nrebpbe5g97sptu/mel_spec_datadict?dl=0
# !wget -O mel_spec_datadict_all https://www.dropbox.com/s/e9l6xid06080g6z/mel_spec_datadict_all?dl=0

--2022-05-04 05:33:10--  https://www.dropbox.com/s/nrebpbe5g97sptu/mel_spec_datadict?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nrebpbe5g97sptu/mel_spec_datadict [following]
--2022-05-04 05:33:11--  https://www.dropbox.com/s/raw/nrebpbe5g97sptu/mel_spec_datadict
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6258da8f6ff8f96f72a5f2ed4.dl.dropboxusercontent.com/cd/0/inline/BkmtCbDUahqFkdk-YW9RPlrrwANNy2yLJKCo6nw2BihwiLVwVdts_-GrmbtCE3GEpVgjWX1KZjXhtB7oXG057-_ZjL0EGnTb6fjGvlpV8XrDPVcM0YbSViVaM9EHjhYzSWMHVIxbVb7AH-eAemJnZRH_1hxDRDVN8kyFBwiSb9bsTw/file# [following]
--2022-05-04 05:33:11--  https://uce6258da8f6ff8f96f72a5f2ed4.dl.dropboxusercontent.com/cd/0/inline/BkmtCbDUahqFkdk-YW9RPlrrwANNy2yLJKCo6nw2BihwiLVw

In [6]:
with open('mel_spec_datadict', 'rb') as f:
# with open('mel_spec_datadict_all', 'rb') as f:
  mel_spec_datadict = pickle.load(f)

In [7]:
!wget -O raw_datadict https://www.dropbox.com/s/uzrtwo4rihx75bg/raw_datadict.pkl_n2k?dl=0
# !wget -O raw_datadict_all https://www.dropbox.com/s/yfi64jhyk9wzv6l/raw_datadict_all?dl=0

--2022-05-04 05:35:08--  https://www.dropbox.com/s/uzrtwo4rihx75bg/raw_datadict.pkl_n2k?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/uzrtwo4rihx75bg/raw_datadict.pkl_n2k [following]
--2022-05-04 05:35:09--  https://www.dropbox.com/s/raw/uzrtwo4rihx75bg/raw_datadict.pkl_n2k
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucec34835896b37686184d6ba974.dl.dropboxusercontent.com/cd/0/inline/BklyCmMrHlPc204UhoWqXF0uaowH_crXGox4pKsuPoCEnb1lgOe6ppRk1tbptat-e04sKQx7KNgo3D6mGnInqZxNft3Si2KlX9ZwlY2jO6CUrQgJU7Sl6bbFvHHQC3ma0ctYbMD0kbgiGIgS9zEKmfQHDxG8g0Hz-zseLiTyXoJvsw/file# [following]
--2022-05-04 05:35:09--  https://ucec34835896b37686184d6ba974.dl.dropboxusercontent.com/cd/0/inline/BklyCmMrHlPc204UhoWqXF0uaowH_crXGox4pKs

In [8]:
# with open('raw_datadict_all', 'rb') as f:
with open('raw_datadict', 'rb') as f:
  raw_datadict = pickle.load(f)

In [32]:
# dataset 1: 0-1 popularity classification
# X_train, y_train = mel_spec_datadict['X_train']
# X_val, y_val = mel_spec_datadict['X_holdout']
# X_test, y_test = mel_spec_datadict['X_test']


# dataset 1: 0-1 popularity classification
X_train = mel_spec_datadict['X_train'][0]
X_val = mel_spec_datadict['X_holdout'][0]
X_test = mel_spec_datadict['X_test'][0]

y_train_dummies = pd.get_dummies(raw_datadict['X_train'][0]["genre"])
y_train_dummies = y_train_dummies.reindex(sorted(y_train_dummies.columns), axis=1)
idx_train = list(set(raw_datadict['X_train'][0]["genre"].index) - set(mel_spec_datadict['X_train'][1].index))
y_train = y_train_dummies[~raw_datadict['X_train'][0].index.isin(idx_train)]

y_val_dummies = pd.get_dummies(raw_datadict['X_holdout'][0]["genre"])
y_val_dummies = y_val_dummies.reindex(sorted(y_val_dummies.columns), axis=1)
y_val = y_val_dummies

y_test_dummies = pd.get_dummies(raw_datadict['X_test'][0]["genre"])
y_test_dummies = y_test_dummies.reindex(sorted(y_test_dummies.columns), axis=1)
idx_test = list(set(raw_datadict['X_test'][0]["genre"].index) - set(mel_spec_datadict['X_test'][1].index))
y_test = y_test_dummies[~raw_datadict['X_test'][0].index.isin(idx_test)]


# dataset 2: 0-1 popularity classification
# X_train = mel_spec_datadict['X_train'][0]
# X_val = mel_spec_datadict['X_holdout'][0]
# X_test = mel_spec_datadict['X_test'][0]
# y_train = raw_datadict['X_train'][0]['popGreaterThan50']
# y_val = raw_datadict['X_holdout'][0]['popGreaterThan50']
# y_test = raw_datadict['X_test'][0]['popGreaterThan50']


# dataset 2: 0-1 popularity regression
# X_train = mel_spec_datadict['X_train'][0]
# X_val = mel_spec_datadict['X_holdout'][0]
# X_test = mel_spec_datadict['X_test'][0]
# y_train = np.round(raw_datadict['X_train'][0]['popularity'] / 20.0) / 5.0
# y_val = np.round(raw_datadict['X_holdout'][0]['popularity'] / 20.0) / 5.0
# y_test = np.round(raw_datadict['X_test'][0]['popularity'] / 20.0) / 5.0

In [33]:
pl.seed_everything(0, workers=True)

Global seed set to 0


0

In [45]:
# hyperparameters
NB_EPOCHS = 30
PATIENCE = 6
DOWNSAMPLING_FACTOR = 10
INPUT_TDIM = int(X_train.shape[2] / DOWNSAMPLING_FACTOR)
INPUT_FDIM = X_train.shape[1]

TASK = 'multiclass' # 'classification', 'regression'
MONITOR = 'val_loss'
MONITOR_HYPERPARAM = 'val_acc' #'val_auc'
BATCH_SIZE = 32
LEARNING_RATE = 4e-5
DROPOUT_PROB = 0.5
USE_CLS_POOLING = True
NB_LAYERS_TO_FREEZE = 6
GRAD_ACC = 1
USE_SIGMOID = True

# TASK = 'classification' # 'multiclass', 'regression'
# MONITOR = 'val_loss'
# MONITOR_HYPERPARAM = 'val_auc' #'val_auc'
# BATCH_SIZE = 32
# LEARNING_RATE = 7.83833546396864e-05
# DROPOUT_PROB = 0.40324782505879203
# USE_CLS_POOLING = True
# NB_LAYERS_TO_FREEZE = 8
# GRAD_ACC = 1
# USE_SIGMOID = True

In [35]:
# downsampling
indices_to_isolate = [k * DOWNSAMPLING_FACTOR for k in range(INPUT_TDIM)]
len(indices_to_isolate)

129

In [36]:
X_train.shape

(1999, 128, 1292, 1)

In [37]:
y_train.shape

(1999, 5)

In [38]:
X_test.shape

(1997, 128, 1292, 1)

In [39]:
y_test.shape

(1997, 5)

In [51]:
# data
X_train_downsampled = X_train[:,:,indices_to_isolate,:]
X_train_normalized = (X_train_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)
X_val_downsampled = X_val[:,:,indices_to_isolate,:]
X_val_normalized = (X_val_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)
X_test_downsampled = X_test[:,:,indices_to_isolate,:]
X_test_normalized = (X_test_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)

X_train_ = torch.from_numpy(X_train_normalized).to(torch.float16).squeeze(-1)
X_val_ = torch.from_numpy(X_val_normalized).to(torch.float16).squeeze(-1)
X_test_ = torch.from_numpy(X_test_normalized).to(torch.float16).squeeze(-1)
y_train_ = torch.from_numpy(y_train.values).to(torch.float16)
y_val_ = torch.from_numpy(y_val.values).to(torch.float16)
y_test_ = torch.from_numpy(y_test.values).to(torch.float16)

#data augmentation
def transform_spec(spec):
  if np.random.random() <= 0.6:
    p = np.random.random()
    if p  <= 0.6:
      mask_param = 40
      transform = nn.Sequential(transforms.FrequencyMasking(freq_mask_param=mask_param), 
                                transforms.TimeMasking(time_mask_param=mask_param)).to(device)
      return transform(spec.to(device))
    elif 0.6 <= p <= 1:
      transform = transforms.Fade(fade_in_len = 50, fade_out_len = 50)
      return transform(spec.to(device))
    else:
      transform = transforms.PitchShift(sample_rate = 10, n_steps = 10, n_fft=128).to(device)
      return transform(spec.to(device))
  else:
    return spec

class SpectDataset(torch.utils.data.TensorDataset):
  # def __getitem__(self, index):
  #       return tuple(tensor[index] for tensor in self.tensors)
  def __getitem__(self, index):
    return tuple(transform_spec(tensor.to(device))[index] if len(tensor.shape) == 3 else tensor.to(device)[index]
                 for tensor in self.tensors)
    
train = SpectDataset(X_train_, y_train_)
val = torch.utils.data.TensorDataset(X_val_, y_val_)
test = torch.utils.data.TensorDataset(X_test_, y_test_)

train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE*2)
test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE*2)

In [64]:
#class to wrap pretrained model
class CustomASTModel(pl.LightningModule):

  def __init__(
      self,
      input_fdim=INPUT_FDIM,
      input_tdim=INPUT_TDIM,
      use_small=True,
      use_cls_pooling=USE_CLS_POOLING,
      dropout_prob=DROPOUT_PROB,
      task=TASK,
      regression_threshold=100.0,
      multiclass_nb_classes=len(y_train.columns) if TASK == 'multiclass' else None,
      nb_layers_to_freeze=NB_LAYERS_TO_FREEZE,
      lr=LEARNING_RATE,
      use_sigmoid=USE_SIGMOID
  ):
    super().__init__()
    self.lr = lr
    self.dropout_prob = dropout_prob
    self.regression_threshold = regression_threshold
    self.use_sigmoid = use_sigmoid
    self.multiclass_nb_classes = multiclass_nb_classes

    if use_cls_pooling:
      self.pooling_type = 'ft_cls'
    else:
      self.pooling_type = 'ft_avgtok'

    assert(task in ['classification','regression','multiclass'])
    self.task = task

    if use_small:
      self.model = ASTModel(
          label_dim=1,    # no impact
          fshape=16, 
          tshape=16, 
          fstride=10, 
          tstride=10,
          input_fdim=input_fdim, 
          input_tdim=input_tdim, 
          model_size='small',
          pretrain_stage=False, 
          load_pretrained_mdl_path=(
              '/content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth'
          )
      )
    else:
       self.model = ASTModel(
          label_dim=1,
          fshape=16, 
          tshape=16, 
          fstride=10, 
          tstride=10,
          input_fdim=input_fdim, 
          input_tdim=input_tdim, 
          model_size='base',
          pretrain_stage=False, 
          load_pretrained_mdl_path=(
              '/content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Base-Patch-400.pth'
          )
       ) 
    self.cls_shape = self.model.v.blocks[-1].mlp.fc2.out_features
    mlp_head = nn.Sequential(
        nn.LayerNorm(self.cls_shape),
        nn.Linear(in_features=self.cls_shape, out_features=int(self.cls_shape / 2), bias=True),
        nn.Dropout(self.dropout_prob),
        nn.Linear(
            in_features=int(self.cls_shape / 2), 
            out_features=(1 if self.task != 'multiclass' else self.multiclass_nb_classes), 
            bias=True
        ),
    )
    self.model.mlp_head = mlp_head

    if nb_layers_to_freeze > 0:
      assert(nb_layers_to_freeze-1 in range(len(self.model.v.blocks)))
      for i,block in enumerate(self.model.v.blocks):
        if i < nb_layers_to_freeze:
          for param in block.parameters():
            param.requires_grad = False
        else:
          break


  def forward(self, x):
    h = self.model(x, task=self.pooling_type)
    return x


  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    return optimizer


  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    x = x.cuda(0)
    y = y.cuda(0).unsqueeze(1)
    y_hat = self.model(x, task=self.pooling_type)

    if self.task == 'classification':
      loss = F.binary_cross_entropy_with_logits(y_hat, y)
      sigmoid = torch.sigmoid(y_hat)
      auc = roc_auc_score(y.cpu().detach().numpy(), sigmoid.cpu().detach().numpy())
      threshold = torch.tensor([0.5]).to(torch.float16).cuda(0)
      results = (sigmoid > threshold).to(torch.float16)
      acc = accuracy_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      f1 = f1_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      self.log('train_auc', auc)
      self.log('train_acc', acc)
      self.log('train_f1', acc)

    elif self.task == 'regression':
      # y_hat = torch.relu(y_hat)   # we are trying to predict something positive so using relu makes sense
      if self.use_sigmoid:
        y_hat = torch.sigmoid(y_hat) * 100.0
      else:
        y_hat = torch.clip(y_hat, min=np.float16(0.0), max=np.float16(self.regression_threshold))

      y_hat = (torch.round(y_hat / 20.0) / 5.0).to(torch.float16)
      loss = F.mse_loss(y_hat, y)
      rmse = np.sqrt(mean_squared_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()))
      mae = mean_absolute_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      self.log('train_rmse', rmse)
      self.log('train_mae', mae)
      self.log('train_r2', r2)

    elif self.task == 'multiclass':
      # criterion = nn.CrossEntropyLoss()
      y = y.squeeze()
      # loss = criterion(y_hat, y)
      loss = F.cross_entropy(y_hat, y)
      y_hat_class = torch.argmax(y_hat, axis = 1)
      y_class = torch.argmax(y, axis = 1)
      acc = accuracy_score(y_class.cpu().detach().numpy(), y_hat_class.cpu().detach().numpy())
      self.log('train_acc', acc)

    self.log('train_loss', loss)
    return loss


  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    x = x.cuda(0)
    y = y.cuda(0).unsqueeze(1)
    y_hat = self.model(x, task=self.pooling_type)

    if self.task == 'classification':
      loss = F.binary_cross_entropy_with_logits(y_hat, y)
      sigmoid = torch.sigmoid(y_hat)
      auc = roc_auc_score(y.cpu().detach().numpy(), sigmoid.cpu().detach().numpy())
      threshold = torch.tensor([0.5]).to(torch.float16).cuda(0)
      results = (sigmoid > threshold).to(torch.float16)
      acc = accuracy_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      f1 = f1_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      self.log('val_auc', auc)
      self.log('val_acc', acc)
      self.log('val_f1', acc)

    elif self.task == 'regression':
      # y_hat = torch.relu(y_hat)   # we are trying to predict something positive so using relu makes sense
      if self.use_sigmoid:
        y_hat = torch.sigmoid(y_hat) * 100.0
      else:
        y_hat = torch.clip(y_hat, min=np.float16(0.0), max=np.float16(self.regression_threshold))
        
      y_hat = (torch.round(y_hat / 20.0) / 5.0).to(torch.float16)
      loss = F.mse_loss(y_hat, y)
      rmse = np.sqrt(mean_squared_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()))
      mae = mean_absolute_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      self.log('val_rmse', rmse)
      self.log('val_mae', mae)
      self.log('val_r2', r2)

    elif self.task == 'multiclass':
      criterion = nn.CrossEntropyLoss()
      y = y.squeeze()
      loss = criterion(y_hat, y)
      # loss = F.cross_entropy(y_hat, y)
      y_hat_class = torch.argmax(y_hat, axis = 1)
      y_class = torch.argmax(y, axis = 1)
      acc = accuracy_score(y_class.cpu().detach().numpy(), y_hat_class.cpu().detach().numpy())
      self.log('val_acc', acc)

    self.log('val_loss', loss)

# some simple tests

In [42]:
# loaders
# train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
# val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE*2)
# test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE*2)

In [59]:
model = CustomASTModel(
    use_cls_pooling=USE_CLS_POOLING,
    dropout_prob=DROPOUT_PROB,
    nb_layers_to_freeze=NB_LAYERS_TO_FREEZE,
    lr=LEARNING_RATE,
    task=TASK,
    use_sigmoid=USE_SIGMOID   
)

early_stop_callback = EarlyStopping(
    monitor=MONITOR, 
    min_delta=0.00, 
    patience=PATIENCE, 
    verbose=True, 
    mode=(
        "max" if MONITOR in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
    )
)

checkpoint_callback = ModelCheckpoint(
    save_top_k=1, 
    monitor=MONITOR_HYPERPARAM, 
    mode=(
        "max" if MONITOR_HYPERPARAM in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
    )
)

trainer = pl.Trainer(
    gpus=1,
    limit_train_batches=1.0, 
    accumulate_grad_batches=GRAD_ACC,
    precision=16, 
    max_epochs=NB_EPOCHS, 
    callbacks=[early_stop_callback, checkpoint_callback],
    enable_checkpointing=True,
    log_every_n_steps=16,
    # enable_progress_bar=False
)

now load a SSL pretrained models from /content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..


pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
fine-tuning patch split stride: frequncey=10, time=10
fine-tuning number of patches=144


In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | ASTModel | 22.3 M
-----------------------------------
11.6 M    Trainable params
10.6 M    Non-trainable params
22.3 M    Total params
44.591    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

x shape:  torch.Size([64, 128, 129])
y shape:  torch.Size([64, 1, 5])
x shape:  torch.Size([64, 128, 129])
y shape:  torch.Size([64, 1, 5])


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [66]:
val_results = trainer.validate(model, val_loader)
# 0.6365%

In [1]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

# hyperparameter tuning

In [61]:
def objective(trial): 

  BATCH_SIZE = trial.suggest_int("batch_size", 1, 4)
  BATCH_SIZE *= 16
  LEARNING_RATE = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
  DROPOUT_PROB = trial.suggest_float("dropout_prob", 0.0, 0.5)
  USE_CLS_POOLING = trial.suggest_categorical("use_cls_pooling", [True, False])
  NB_LAYERS_TO_FREEZE = trial.suggest_int("nb_layers_to_freeze", 0, 12)
  GRAD_ACC = trial.suggest_int("grad_acc", 1, 2)
  # USE_SIGMOID = trial.suggest_categorical("use_sigmoid", [True, False])
  USE_SIGMOID = True

  train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
  val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE)
  test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE)

  model = CustomASTModel(
      use_cls_pooling=USE_CLS_POOLING,
      dropout_prob=DROPOUT_PROB,
      nb_layers_to_freeze=NB_LAYERS_TO_FREEZE,
      lr=LEARNING_RATE,
      task=TASK,
      use_sigmoid=USE_SIGMOID   
  )

  early_stop_callback = EarlyStopping(
      monitor=MONITOR, 
      min_delta=0.00, 
      patience=PATIENCE, 
      verbose=True, 
      mode=(
          "max" if MONITOR in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
      )
  )

  checkpoint_callback = ModelCheckpoint(
      save_top_k=1, 
      monitor=MONITOR_HYPERPARAM, 
      mode=(
          "max" if MONITOR_HYPERPARAM in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
      )
  )

  trainer = pl.Trainer(
      gpus=1,
      limit_train_batches=1.0, 
      accumulate_grad_batches=GRAD_ACC,
      precision=16, 
      max_epochs=NB_EPOCHS, 
      callbacks=[early_stop_callback, checkpoint_callback],
      enable_checkpointing=True,
      log_every_n_steps=16,
      # enable_progress_bar=False
  )

  trainer.fit(model, train_loader, val_loader)

  # val_results = trainer.validate(model, val_loader)

  # return val_results[0][MONITOR_HYPERPARAM]
  return checkpoint_callback.best_model_score

In [2]:
study = optuna.create_study(
    direction="maximize" if MONITOR_HYPERPARAM in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "minimize"
)
study.optimize(objective, n_trials=15, gc_after_trial=True)

In [3]:
study.best_trial

In [ ]:
# first dataset
'''
AUC: 0.7211533030146442
{
  'batch_size': 2, 
  'learning_rate': 7.83833546396864e-05, 
  'dropout_prob': 0.40324782505879203, 
  'use_cls_pooling': True, 
  'nb_layers_to_freeze': 8, 
  'grad_acc': 1
}
'''